# Selecting best features

In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# automated feature engineering
import featuretools as ft

# Filter out pandas warnings
import warnings 
warnings.filterwarnings('ignore')

import sys
sys.path.append('../')
from helpers import *

In [35]:
pd.options.display.max_rows, pd.options.display.max_columns

# https://thispointer.com/python-pandas-how-to-display-full-dataframe-i-e-print-all-rows-columns-without-truncation/
# Print all the contents of a pandas dataframe
pd.set_option(
    "display.max_rows", None
)  # Print unlimited number of rows by setting to None, default is 10
pd.set_option(
    "display.max_columns", None
)  # Do not truncate columns to display all of them by setting to None
pd.set_option(
    "display.width", None
)  # Auto-detect the width of dataframe to display all columns in single line by setting to None
pd.set_option(
    "display.max_colwidth", None
)  # Auto detect the max size of column and print contents of that column without truncation

In [36]:
app_train = pd.read_csv('../data/application_train.csv').sort_values('SK_ID_CURR').reset_index().loc[:1000, :].drop(columns = ['index'])
app_test = pd.read_csv('../data/application_test.csv').sort_values('SK_ID_CURR').reset_index().loc[:1000, :].drop(columns = ['index'])
bureau = pd.read_csv('../data/bureau.csv').sort_values(['SK_ID_CURR', 'SK_ID_BUREAU']).reset_index().loc[:1000, :].drop(columns = ['index'])
bureau_balance = pd.read_csv('../data/bureau_balance.csv').sort_values('SK_ID_BUREAU').reset_index().loc[:1000, :].drop(columns = ['index'])
cash = pd.read_csv('../data/POS_CASH_balance.csv').sort_values(['SK_ID_CURR', 'SK_ID_PREV']).reset_index().loc[:1000, :].drop(columns = ['index'])
credit = pd.read_csv('../data/credit_card_balance.csv').sort_values(['SK_ID_CURR', 'SK_ID_PREV']).reset_index().loc[:1000, :].drop(columns = ['index'])
previous = pd.read_csv('../data/previous_application.csv').sort_values(['SK_ID_CURR', 'SK_ID_PREV']).reset_index().loc[:1000, :].drop(columns = ['index'])
installments = pd.read_csv('../data/installments_payments.csv').sort_values(['SK_ID_CURR', 'SK_ID_PREV']).reset_index().loc[:1000, :].drop(columns = ['index'])


In [37]:
def replace_day_outliers(df):
    """Replace 365243 with np.nan in any columns with DAYS"""
    for col in df.columns:
        if "DAYS" in col:
            df[col] = df[col].replace({365243: np.nan})

    return df

# Replace all the day outliers
app_train = replace_day_outliers(app_train)
app_test = replace_day_outliers(app_test)
bureau = replace_day_outliers(bureau)
bureau_balance = replace_day_outliers(bureau_balance)
credit = replace_day_outliers(credit)
cash = replace_day_outliers(cash)
previous = replace_day_outliers(previous)
installments = replace_day_outliers(installments)


In [39]:
# Add identifying column
app_train['set'] = 'train'
app_test['set'] = 'test'
app_test["TARGET"] = np.nan

# Append the dataframes
app = app_train.append(app_test, ignore_index = True)

grouped = bureau.groupby('SK_ID_CURR')
app['debt_credit_ratio_None'] = grouped['AMT_CREDIT_SUM_DEBT'].sum() / grouped['AMT_CREDIT_SUM'].sum()
app['credit_annuity_ratio'] = app['AMT_CREDIT'] / app['AMT_ANNUITY']
prev_sorted = previous.sort_values(by=['SK_ID_CURR', 'DAYS_DECISION'])
app['prev_PRODUCT_COMBINATION'] = prev_sorted.groupby('SK_ID_CURR')['PRODUCT_COMBINATION'].last()
app['DAYS_CREDIT_mean'] = bureau.groupby('SK_ID_CURR')['DAYS_CREDIT'].mean()
app['credit_goods_price_ratio'] = app['AMT_CREDIT'] / app['AMT_GOODS_PRICE']
active_loans = bureau[bureau['CREDIT_ACTIVE'] == 'Active']
app['last_active_DAYS_CREDIT'] = active_loans.groupby('SK_ID_CURR')['DAYS_CREDIT'].max()
app['credit_downpayment'] = app['AMT_GOODS_PRICE'] - app['AMT_CREDIT']
app['AGE_INT'] = (app['DAYS_BIRTH'] / -365).astype(int)

installments['diff'] = installments['AMT_PAYMENT'] - installments['AMT_INSTALMENT']
filtered = installments[installments['DAYS_INSTALMENT'] > -1000]
grouped = filtered.groupby(['SK_ID_PREV', 'SK_ID_CURR'])['diff'].mean().groupby('SK_ID_CURR').mean()

app['installment_payment_ratio_1000_mean_mean'] = grouped
max_installment = installments.groupby('SK_ID_CURR')['AMT_INSTALMENT'].max()
app['annuity_to_max_installment_ratio'] = app['AMT_ANNUITY'] / max_installment


es = ft.EntitySet(id = 'clients')
# For dataframes with an existing unique index
es = es.add_dataframe(dataframe_name='app', dataframe=app, index='SK_ID_CURR')
es = es.add_dataframe(dataframe_name='bureau', dataframe=bureau, index='SK_ID_BUREAU')
es = es.add_dataframe(dataframe_name='previous', dataframe=previous, index='SK_ID_PREV')

# For dataframes that do not have a unique index, manually add an index column
bureau_balance['bureaubalance_index'] = range(1, len(bureau_balance) + 1)
cash['cash_index'] = range(1, len(cash) + 1)
installments['installments_index'] = range(1, len(installments) + 1)
credit['credit_index'] = range(1, len(credit) + 1)

# Now add these dataframes to the EntitySet
es = es.add_dataframe(dataframe_name='bureau_balance', dataframe=bureau_balance, index='bureaubalance_index')
es = es.add_dataframe(dataframe_name='cash', dataframe=cash, index='cash_index')
es = es.add_dataframe(dataframe_name='installments', dataframe=installments, index='installments_index')
es = es.add_dataframe(dataframe_name='credit', dataframe=credit, index='credit_index')

# Define relationships based on logical connections (foreign keys) between the dataframes
relationships = [
    ('app', 'SK_ID_CURR', 'bureau', 'SK_ID_CURR'),
    ('bureau', 'SK_ID_BUREAU', 'bureau_balance', 'SK_ID_BUREAU'),
    ('app', 'SK_ID_CURR', 'previous', 'SK_ID_CURR'),
    ('previous', 'SK_ID_PREV', 'cash', 'SK_ID_PREV'),
    ('previous', 'SK_ID_PREV', 'installments', 'SK_ID_PREV'),
    ('previous', 'SK_ID_PREV', 'credit', 'SK_ID_PREV')
]

# Add relationships to the EntitySet
for parent_df, parent_col, child_df, child_col in relationships:
    es.add_relationship(parent_dataframe_name=parent_df, parent_column_name=parent_col,
                        child_dataframe_name=child_df, child_column_name=child_col)


In [40]:
es

Entityset: clients
  DataFrames:
    app [Rows: 2002, Columns: 133]
    bureau [Rows: 1001, Columns: 17]
    previous [Rows: 1001, Columns: 37]
    bureau_balance [Rows: 1001, Columns: 4]
    cash [Rows: 1001, Columns: 9]
    installments [Rows: 1001, Columns: 10]
    credit [Rows: 1001, Columns: 24]
  Relationships:
    bureau.SK_ID_CURR -> app.SK_ID_CURR
    bureau_balance.SK_ID_BUREAU -> bureau.SK_ID_BUREAU
    previous.SK_ID_CURR -> app.SK_ID_CURR
    cash.SK_ID_PREV -> previous.SK_ID_PREV
    installments.SK_ID_PREV -> previous.SK_ID_PREV
    credit.SK_ID_PREV -> previous.SK_ID_PREV

### Feature Primitives


In [41]:
primitives = ft.list_primitives()
pd.options.display.max_colwidth = 100
primitives[primitives['type'] == 'aggregation'].head(10)
primitives[(primitives['type'] == 'aggregation') & (primitives['dask_compatible'] == True)].name.to_list
primitives[(primitives['type'] == 'transform') & (primitives['dask_compatible'] == True)].name.to_list()


['sine',
 'multiply_numeric_scalar',
 'day_of_year',
 'less_than_scalar',
 'is_weekend',
 'cosine',
 'greater_than_equal_to_scalar',
 'is_month_end',
 'month',
 'two_digit_postal_code',
 'multiply_boolean',
 'square_root',
 'equal',
 'days_in_month',
 'and',
 'modulo_by_feature',
 'num_characters',
 'subtract_numeric',
 'not_equal',
 'greater_than_scalar',
 'is_month_start',
 'second',
 'is_working_hours',
 'tangent',
 'multiply_numeric',
 'divide_by_feature',
 'part_of_day',
 'not_equal_scalar',
 'age',
 'day',
 'less_than',
 'week',
 'is_quarter_end',
 'natural_logarithm',
 'is_leap_year',
 'is_year_end',
 'subtract_numeric_scalar',
 'num_words',
 'modulo_numeric',
 'not',
 'less_than_equal_to',
 'is_quarter_start',
 'equal_scalar',
 'weekday',
 'hour',
 'add_numeric',
 'or',
 'is_year_start',
 'negate',
 'divide_numeric',
 'quarter',
 'year',
 'greater_than',
 'time_since',
 'modulo_numeric_scalar',
 'absolute',
 'is_lunch_time',
 'multiply_numeric_boolean',
 'less_than_equal_to_sca

In [42]:
default_agg_primitives =  ["sum", "std", "max", "skew", "min", "mean", "count", "percent_true", "num_unique", "mode"]
default_trans_primitives =  ["day", "year", "month", "weekday", "haversine", "num_characters"]

In [ ]:
import gc
gc.enable()
del bureau_balance, bureau, credit, installments, cash, previous 
gc.collect()

158

In [ ]:
# DFS with default primitives

feature_matrix, feature_names = ft.dfs(entityset = es, target_dataframe_name = 'app',
                                       trans_primitives = default_trans_primitives,
                                       agg_primitives=default_agg_primitives, 
                                        max_depth = 2, features_only=False, verbose = True)
with open('features_data/feature_names.txt', 'w') as f:
    for item in feature_names:
        f.write("%s\n" % item)
feature_matrix.to_csv('features_data/feature_matrix.csv')

['<Feature: STD(credit.previous.SELLERPLACE_AREA)>',
 '<Feature: SUM(credit.previous.AMT_ANNUITY)>',
 '<Feature: SUM(credit.previous.AMT_APPLICATION)>',
 '<Feature: SUM(credit.previous.AMT_CREDIT)>',
 '<Feature: SUM(credit.previous.AMT_DOWN_PAYMENT)>',
 '<Feature: SUM(credit.previous.AMT_GOODS_PRICE)>',
 '<Feature: SUM(credit.previous.CNT_PAYMENT)>',
 '<Feature: SUM(credit.previous.DAYS_DECISION)>',
 '<Feature: SUM(credit.previous.DAYS_FIRST_DRAWING)>',
 '<Feature: SUM(credit.previous.DAYS_FIRST_DUE)>',
 '<Feature: SUM(credit.previous.DAYS_LAST_DUE)>',
 '<Feature: SUM(credit.previous.DAYS_LAST_DUE_1ST_VERSION)>',
 '<Feature: SUM(credit.previous.DAYS_TERMINATION)>',
 '<Feature: SUM(credit.previous.HOUR_APPR_PROCESS_START)>',
 '<Feature: SUM(credit.previous.NFLAG_INSURED_ON_APPROVAL)>',
 '<Feature: SUM(credit.previous.NFLAG_LAST_APPL_IN_DAY)>',
 '<Feature: SUM(credit.previous.RATE_DOWN_PAYMENT)>',
 '<Feature: SUM(credit.previous.RATE_INTEREST_PRIMARY)>',
 '<Feature: SUM(credit.previous.R

TARGET NAME_CONTRACT_TYPE CODE_GENDER FLAG_OWN_CAR FLAG_OWN_REALTY  \
0       1         Cash loans           M            N               Y   
1       0         Cash loans           F            N               N   
2       0    Revolving loans           M            Y               Y   
3       0         Cash loans           F            N               Y   
4       0         Cash loans           M            N               Y   
5       0         Cash loans           M            N               Y   
6       0         Cash loans           F            Y               Y   
7       0         Cash loans           M            Y               Y   
8       0         Cash loans           F            N               Y   
9       0    Revolving loans           M            N               Y   

   CNT_CHILDREN  AMT_INCOME_TOTAL  AMT_CREDIT  AMT_ANNUITY  AMT_GOODS_PRICE  \
0             0          202500.0    406597.5      24700.5         351000.0   
1             0          270000.0   1293502.5      35698.5        1129500.0   
2             0           67500.0    135000.0       6750.0         135000.0   
3             0          135000.0    312682.5      29686.5         297000.0   
4             0          121500.0    513000.0      21865.5         513000.0   
5             0           99000.0    490495.5      27517.5         454500.0   
6             1          171000.0   1560726.0      41301.0        1395000.0   
7             0          360000.0   1530000.0      42075.0        1530000.0   
8             0          112500.0   1019610.0      33826.5         913500.0   
9             0          135000.0    405000.0      20250.0         405000.0   

   NAME_TYPE_SUITE      NAME_INCOME_TYPE            NAME_EDUCATION_TYPE  \
0    Unaccompanied               Working  Secondary / secondary special   
1           Family         State servant               Higher education   
2    Unaccompanied               Working  Secondary / secondary special   
3    Unaccompanied               Working  Secondary / secondary special   
4    Unaccompanied               Working  Secondary / secondary special   
5  Spouse, partner         State servant  Secondary / secondary special   
6    Unaccompanied  Commercial associate               Higher education   
7    Unaccompanied         State servant               Higher education   
8         Children             Pensioner  Secondary / secondary special   
9    Unaccompanied               Working  Secondary / secondary special   

     NAME_FAMILY_STATUS  NAME_HOUSING_TYPE  REGION_POPULATION_RELATIVE  \
0  Single / not married  House / apartment                    0.018801   
1               Married  House / apartment                    0.003541   
2  Single / not married  House / apartment                    0.010032   
3        Civil marriage  House / apartment                    0.008019   
4  Single / not married  House / apartment                    0.028663   
5               Married  House / apartment                    0.035792   
6               Married  House / apartment                    0.035792   
7               Married  House / apartment                    0.003122   
8               Married  House / apartment                    0.018634   
9  Single / not married  House / apartment                    0.019689   

   DAYS_BIRTH  DAYS_EMPLOYED  DAYS_REGISTRATION  DAYS_ID_PUBLISH  OWN_CAR_AGE  \
0       -9461           -637            -3648.0            -2120          NaN   
1      -16765          -1188            -1186.0             -291          NaN   
2      -19046           -225            -4260.0            -2531         26.0   
3      -19005          -3039            -9833.0            -2437          NaN   
4      -19932          -3038            -4311.0            -3458          NaN   
5      -16941          -1588            -4970.0             -477          NaN   
6      -13778          -3130            -1213.0             -619         17.0   
7      -18850           -449            